# Preparing Rental Data

### Scarped data with scraper.py and now preparing it to load to database.

## Import Libraries

In [13]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
import os
import hashlib
import re

## Load Rental Dataset

In [14]:
root = tk.Tk()
root.withdraw()
csv_path = filedialog.askopenfilename(title="Select CSV file", filetypes=[("CSV files", "*.csv")])
print(os.path.basename(csv_path))
'''
csv_path = 'file_name.csv'
'''
df = pd.read_csv(csv_path)
df.head()

san_diego_county_rentals_2025-05-31.csv


,Property,Address,City,State,ZipCode,Phone,Unit,Beds,Baths,Beds_Baths,...,PricePerSqFt,RentalType,HasWasherDryer,HasAirConditioning,HasPool,HasSpa,HasGym,HasEVCharging,IsPetFriendly,ListingURL
0,Radian,"675 9th Ave, San Diego, CA 92101",San Diego,CA,92101,8582216613,Unit 1205,1,1.0,1 Bed / 1 Bath,...,4.68,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
1,Radian,"675 9th Ave, San Diego, CA 92101",San Diego,CA,92101,8582216613,Unit 1405,1,1.0,1 Bed / 1 Bath,...,4.72,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
2,Radian,"675 9th Ave, San Diego, CA 92101",San Diego,CA,92101,8582216613,Unit 1305,1,1.0,1 Bed / 1 Bath,...,4.70,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
3,Radian,"675 9th Ave, San Diego, CA 92101",San Diego,CA,92101,8582216613,Unit 1413,1,1.0,1 Bed / 1 Bath,...,4.77,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
4,Radian,"675 9th Ave, San Diego, CA 92101",San Diego,CA,92101,8582216613,Unit 1513,1,1.0,1 Bed / 1 Bath,...,4.81,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...


### Address & Unit Standardization

In [15]:
def smart_address_title(s):
    if pd.isnull(s):
        return s
    s = str(s).strip().title()
    # Lowercase ordinal suffixes like '9Th' → '9th', '2Nd' → '2nd'
    s = re.sub(r'(\d+)(St|Nd|Rd|Th)\b', lambda m: m.group(1) + m.group(2).lower(), s)
    return s

for col in ['Address', 'Unit']:
    df[col] = df[col].apply(smart_address_title)

# SqFt should stay as number or as-is, not title-cased!
df['SqFt'] = df['SqFt'].astype(str).str.strip()

print(df.head())

  Property                           Address       City State  ZipCode  \
0   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA    92101   
1   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA    92101   
2   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA    92101   
3   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA    92101   
4   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA    92101   

        Phone       Unit  Beds  Baths      Beds_Baths  ... PricePerSqFt  \
0  8582216613  Unit 1205     1    1.0  1 Bed / 1 Bath  ...         4.68   
1  8582216613  Unit 1405     1    1.0  1 Bed / 1 Bath  ...         4.72   
2  8582216613  Unit 1305     1    1.0  1 Bed / 1 Bath  ...         4.70   
3  8582216613  Unit 1413     1    1.0  1 Bed / 1 Bath  ...         4.77   
4  8582216613  Unit 1513     1    1.0  1 Bed / 1 Bath  ...         4.81   

   RentalType  HasWasherDryer HasAirConditioning  HasPool  HasSpa  HasGym  \
0   Apartment            Tr

### Create Property Key

In [16]:
# Create unique property_key
df['property_key'] = df['Address'] + '|' + df['Unit'] + '|' + df['SqFt']
print(f"Unique properties: {df['property_key'].nunique()}")

Unique properties: 4839


### Create Property ID

In [17]:
# Deterministic 12-digit property_id from property_key
def deterministic_12_digit(s):
    h = int(hashlib.sha256(s.encode()).hexdigest(), 16)
    return str(h)[-12:]

df['property_id'] = df['property_key'].apply(deterministic_12_digit)

# Drop the temporary property_key
df.drop(columns=['property_key'], inplace=True)

# Move property_id to the first column
cols = list(df.columns)
cols.insert(0, cols.pop(cols.index('property_id')))
df = df[cols]

## Preview the result

In [18]:
# Preview the result
df.head()

,property_id,Property,Address,City,State,ZipCode,Phone,Unit,Beds,Baths,...,PricePerSqFt,RentalType,HasWasherDryer,HasAirConditioning,HasPool,HasSpa,HasGym,HasEVCharging,IsPetFriendly,ListingURL
0,346891743979,Radian,"675 9th Ave, San Diego, Ca 92101",San Diego,CA,92101,8582216613,Unit 1205,1,1.0,...,4.68,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
1,448601032798,Radian,"675 9th Ave, San Diego, Ca 92101",San Diego,CA,92101,8582216613,Unit 1405,1,1.0,...,4.72,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
2,546388333680,Radian,"675 9th Ave, San Diego, Ca 92101",San Diego,CA,92101,8582216613,Unit 1305,1,1.0,...,4.70,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
3,201052451271,Radian,"675 9th Ave, San Diego, Ca 92101",San Diego,CA,92101,8582216613,Unit 1413,1,1.0,...,4.77,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...
4,315344383483,Radian,"675 9th Ave, San Diego, Ca 92101",San Diego,CA,92101,8582216613,Unit 1513,1,1.0,...,4.81,Apartment,True,True,True,True,True,False,True,https://www.apartments.com/radian-san-diego-ca...


In [19]:
df.shape

(14087, 23)

## Remove Duplicates

In [20]:
# Keep only the first occurrence of each property_id
df = df.drop_duplicates(subset='property_id', keep='first').reset_index(drop=True)
print(f"Unique property_id count: {df['property_id'].nunique()}")

Unique property_id count: 4839


### Add uniform month and year of data pull

In [21]:
import tkinter as tk
from tkinter import ttk

# Create dropdown dialog for month and year
def ask_month_year():
    dialog = tk.Toplevel()
    dialog.title("Select Month and Year")
    dialog.grab_set()

    tk.Label(dialog, text="Month:").grid(row=0, column=0, padx=5, pady=5)
    tk.Label(dialog, text="Year:").grid(row=1, column=0, padx=5, pady=5)

    months = [
        "January", "February", "March", "April", "May", "June",
        "July", "August", "September", "October", "November", "December"
    ]
    years = [str(y) for y in range(2020, 2031)]

    month_var = tk.StringVar(value=month.capitalize())
    year_var = tk.StringVar(value=year)

    month_cb = ttk.Combobox(dialog, textvariable=month_var, values=months, state="readonly")
    year_cb = ttk.Combobox(dialog, textvariable=year_var, values=years, state="readonly")
    month_cb.grid(row=0, column=1, padx=5, pady=5)
    year_cb.grid(row=1, column=1, padx=5, pady=5)

    def on_ok():
        dialog.result = (month_cb.get(), year_cb.get())
        dialog.destroy()

    ok_btn = tk.Button(dialog, text="OK", command=on_ok)
    ok_btn.grid(row=2, column=0, columnspan=2, pady=10)

    dialog.wait_window()
    return getattr(dialog, 'result', (month.capitalize(), year))

selected_month, selected_year = ask_month_year()
df['month'] = selected_month
df['year'] = selected_year
print(df[['month', 'year']].head())

  month  year
0  June  2025
1  June  2025
2  June  2025
3  June  2025
4  June  2025


In [22]:
# Ask user for save location and filename using tkinter
save_path = filedialog.asksaveasfilename(
    title="Save CSV file",
    defaultextension=".csv",
    initialfile="May_2025_SD_county.csv",
    filetypes=[("CSV files", "*.csv")]
)
if save_path:
    df.to_csv(save_path, index=False)
    print(f"Saved to {os.path.basename(save_path)}")
else:
    print("Save cancelled.")

Saved to June_2025_SD_county.csv


In [23]:
print(df.head())

    property_id Property                           Address       City State  \
0  346891743979   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA   
1  448601032798   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA   
2  546388333680   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA   
3  201052451271   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA   
4  315344383483   Radian  675 9th Ave, San Diego, Ca 92101  San Diego    CA   

   ZipCode       Phone       Unit  Beds  Baths  ... HasWasherDryer  \
0    92101  8582216613  Unit 1205     1    1.0  ...           True   
1    92101  8582216613  Unit 1405     1    1.0  ...           True   
2    92101  8582216613  Unit 1305     1    1.0  ...           True   
3    92101  8582216613  Unit 1413     1    1.0  ...           True   
4    92101  8582216613  Unit 1513     1    1.0  ...           True   

  HasAirConditioning  HasPool  HasSpa HasGym  HasEVCharging  IsPetFriendly  \
0               True     T